In [4]:
from IPython.display import display
import ipywidgets as widgets
import time
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [5]:
class Tablero:
    def __init__(self, tamano_celda=(50, 50), n_celdas=(5,5)):
        self.out = widgets.HTML()
        display(self.out)
        self.tamano_celda = tamano_celda
        self.n_celdas = n_celdas
        self.casillas = []

    def get_nceldas(self):
        return self.n_celdas

    def get_casillas(self):
        return self.casillas

    def dibujar(self, objetos):
        tablero = "<table border='1' >{}</table>"
        filas = ""
        for i in range(self.n_celdas[0]):
            s = ""
            for j in range(self.n_celdas[1]):
                contenido =""
                self.casillas.append([i, j])
                for o in objetos:
                    if o.x == j and o.y == i:
                        contenido = \
                            "<div style='transform: rotate({angulo}deg);font-size:{tamano_emoticon}px;'>{emoticon}</div>". \
                                format(angulo = o.angulo, tamano_emoticon = o.tamano_emoticon, emoticon = o.emoticon)
                s += "<td style='height:{alto}px;width:{ancho}px'>{contenido}</td>". \
                    format(alto = self.tamano_celda[0], ancho = self.tamano_celda[1],
                           contenido = contenido)
            filas += "<tr>{}</tr>".format(s)
        tablero = tablero.format(filas)
        self.out.value = tablero

In [6]:
class Agente():
    def __init__(self, x=0, y=0, angulo=0, emoticon="🤖", tamano_emoticon=30):
        self.x = x
        self.y = y
        self.angulo = angulo
        self.emoticon = emoticon
        self.tamano_emoticon = tamano_emoticon
        

In [7]:
class AgenteBasura(Agente):
    def __init__(self, x=0, y=0, angulo=0, emoticon="🍂", tamano_emoticon=30 ):
        super().__init__(x, y, angulo, emoticon, tamano_emoticon)

In [8]:
def GenerarBasura(cantidad, tablero=Tablero()):
    basuras = []
    while len(basuras)<cantidad:
        x = np.random.randint(tablero.get_nceldas()[0])
        y = np.random.randint(tablero.get_nceldas()[0])
        if not any(basura.x == x and basura.y == y for basura in basuras):
            basuras.append(AgenteBasura(x, y))
    return basuras

HTML(value='')

In [9]:
class AgenteAspiradoraReflejo(Agente):
    def __init__(self, x=0, y=0, angulo=0, emoticon="🤖", tamano_emoticon=30, tablero = Tablero()):
        super().__init__(x, y, angulo, emoticon, tamano_emoticon)
        self.tablero = tablero
        self.energia = 100
        
    def dec_energy(self):
        self.energia = self.energia - 1

    def abajo(self):
        if self.y < self.tablero.get_nceldas()[1]-1 :
            self.y += 1
            self.dec_energy()
    
    def derecha(self):
        if self.x < self.tablero.get_nceldas()[0]-1:
            self.x += 1
            self.dec_energy()
    
    def arriba(self):
        if self.y > 0 :
            self.y -= 1
            self.dec_energy()
    
    def izquierda(self):
        if self.x > 0 :
            self.x -= 1
            self.dec_energy()

    def detectar_basura(self, basuras):
        basura_arriba = any(basura.x == self.x and basura.y == self.y - 1 for basura in basuras)
        basura_abajo = any(basura.x == self.x and basura.y == self.y + 1 for basura in basuras)
        basura_izquierda = any(basura.x == self.x - 1 and basura.y == self.y for basura in basuras)
        basura_derecha = any(basura.x == self.x + 1 and basura.y == self.y for basura in basuras)
    
        if basura_derecha:
            self.derecha()
            return True
        elif basura_izquierda:
            self.izquierda()
            return True
        elif basura_arriba:
            self.arriba()
            return True
        elif basura_abajo:
            self.abajo()
            return True
        return False

    def limpiar(self, basuras):
        founded_basura = filter(lambda basura: (basura.x == self.x and basura.y == self.y ), basuras)
        basura = list(founded_basura)
        if basura:
            basuras.remove(basura[0])
            return True
        return False

HTML(value='')

In [12]:
def simulacionReflejo(sleep_time=0.5):
    escenario = Tablero()
    basuras = GenerarBasura(cantidad=10)
    aspiradora = AgenteAspiradoraReflejo()
    movimientos = [aspiradora.derecha, aspiradora.izquierda, aspiradora.arriba, aspiradora.abajo]
    while aspiradora.energia > 0 and len(basuras) != 0:
        aspiradora.limpiar(basuras)
        objetos = basuras + [aspiradora]
        escenario.dibujar(objetos)
        time.sleep(sleep_time)
        if aspiradora.energia > 0:
            if not aspiradora.detectar_basura(basuras):
                np.random.choice(movimientos)()
        else:
            break
        escenario.dibujar(objetos)
        time.sleep(sleep_time)

In [13]:
simulacionReflejo()

HTML(value='')